# CRAWL STOCK API

## Nguồn dữ liệu: DStock, FireAnt

## Phần 1: Quy Trình Khoa học Dữ liệu

### 1. Chuẩn bị dữ liệu

Import các thư viện cần thiết cho việc thu thập dữ liệu.

In [ ]:
import requests
import pandas as pd

Lấy thông tin các công ty đã từng được đưa lên sàn chứng khoán Việt Nam

In [ ]:
def get_stock_list():
    VNDIRECT_API = 'https://finfo-api.vndirect.com.vn/v4/stocks'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
        'Content-Type': 'application/json',
    }
    
    params = {
        "q": "type:stock,ifc~floor:HOSE,HNX,UPCOM",
        "size": "9999"
    }
    
    response = requests.get(url = VNDIRECT_API, params = params, headers = headers)
    
    if response.status_code == 200:
        field_to_get = ["code", "type", "status", "companyName", "listedDate", "delistedDate"]
        df = pd.DataFrame(response.json()["data"])
        df = df[field_to_get]
        return pd.DataFrame(df)
        
    return []

In [ ]:
stock_data = get_stock_list()
stock_data.head()

Data clean up

In [ ]:
import re

def cleanRoman(text):
    pattern = r'\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\b\.?'
    return re.sub(pattern, '', text)

def cleanText(text):
    pattern = r'[^A-Za-z]+'
    return re.sub(pattern, '', text)

def cleanBullet(text):
    pattern = '\w[.)]\s*'
    return re.sub(pattern, '', text)

def removeVietNameAccent(s):
    s = re.sub(r'[àáạảãâầấậẩẫăằắặẳẵ]', 'a', s)
    s = re.sub(r'[ÀÁẠẢÃĂẰẮẶẲẴÂẦẤẬẨẪ]', 'A', s)
    s = re.sub(r'[èéẹẻẽêềếệểễ]', 'e', s)
    s = re.sub(r'[ÈÉẸẺẼÊỀẾỆỂỄ]', 'E', s)
    s = re.sub(r'[òóọỏõôồốộổỗơờớợởỡ]', 'o', s)
    s = re.sub(r'[ÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠ]', 'O', s)
    s = re.sub(r'[ìíịỉĩ]', 'i', s)
    s = re.sub(r'[ÌÍỊỈĨ]', 'I', s)
    s = re.sub(r'[ùúụủũưừứựửữ]', 'u', s)
    s = re.sub(r'[ƯỪỨỰỬỮÙÚỤỦŨ]', 'U', s)
    s = re.sub(r'[ỳýỵỷỹ]', 'y', s)
    s = re.sub(r'[ỲÝỴỶỸ]', 'Y', s)
    s = re.sub(r'[Đ]', 'D', s)
    s = re.sub(r'[đ]', 'd', s)
    return s

def removeSpace(text):
    pattern = r'\s*'
    return re.sub(pattern, '', text)

Lấy các chỉ số báo cáo tài chính

In [ ]:
import datetime

def get_balance_sheet(symbol = "VNM", fromYear = 2021, toYear = 2022):
    companyBalanceSheet = []
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    }
    fireant_bearer_token = "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSIsImtpZCI6IkdYdExONzViZlZQakdvNERWdjV4QkRITHpnSSJ9.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4iLCJhdWQiOiJodHRwczovL2FjY291bnRzLmZpcmVhbnQudm4vcmVzb3VyY2VzIiwiZXhwIjoxOTM5NDc0NDY3LCJuYmYiOjE2Mzk0NzQ0NjcsImNsaWVudF9pZCI6ImZpcmVhbnQudHJhZGVzdGF0aW9uIiwic2NvcGUiOlsib3BlbmlkIiwicHJvZmlsZSIsInJvbGVzIiwiZW1haWwiLCJhY2NvdW50cy1yZWFkIiwiYWNjb3VudHMtd3JpdGUiLCJvcmRlcnMtcmVhZCIsIm9yZGVycy13cml0ZSIsImNvbXBhbmllcy1yZWFkIiwiaW5kaXZpZHVhbHMtcmVhZCIsImZpbmFuY2UtcmVhZCIsInBvc3RzLXdyaXRlIiwicG9zdHMtcmVhZCIsInN5bWJvbHMtcmVhZCIsInVzZXItZGF0YS1yZWFkIiwidXNlci1kYXRhLXdyaXRlIiwidXNlcnMtcmVhZCIsInNlYXJjaCIsImFjYWRlbXktcmVhZCIsImFjYWRlbXktd3JpdGUiLCJibG9nLXJlYWQiLCJpbnZlc3RvcGVkaWEtcmVhZCJdLCJzdWIiOiJkM2UxY2I4MC0xMDc0LTRhMjItYWY4Ny0yNjlhOGM3Mzc2NmMiLCJhdXRoX3RpbWUiOjE2Mzk0NzQ0NjcsImlkcCI6Ikdvb2dsZSIsIm5hbWUiOiJtaW5odHJpLm1pbmh6enh6eEBnbWFpbC5jb20iLCJzZWN1cml0eV9zdGFtcCI6ImIzNDM3MmFkLTgxZjktNGUyYy04NTc4LTBmYWE3NmIxYmMzOSIsInByZWZlcnJlZF91c2VybmFtZSI6Im1pbmh0cmkubWluaHp6eHp4QGdtYWlsLmNvbSIsInVzZXJuYW1lIjoibWluaHRyaS5taW5oenp4enhAZ21haWwuY29tIiwiZnVsbF9uYW1lIjoiTWluaCBUcmkgTmd1eWVuIiwiZW1haWwiOiJtaW5odHJpLm1pbmh6enh6eEBnbWFpbC5jb20iLCJlbWFpbF92ZXJpZmllZCI6InRydWUiLCJqdGkiOiIzY2FjMTQwZGIxMTRkNGMwOWI2MWJjNTA1NmQ0MDg0OCIsImFtciI6WyJleHRlcm5hbCJdfQ.X9deVcDttd06BxdZC7uOBXeObi3qOYqIsWK190UXRBSbVw-03W4KlsQ5PwKyoAc5beog9zYTtZzoE63cnbJ4o14aq4ljsM4bcFEfP2wLl3taVjuKbJOKaFMLiUFyQGiPc5_iE7b-7Z3cVWyEWtDl9xeqg57vVrBLXvcyzquWTFVKgaumR7PA3EwM5UHQWL8f2nx_zwAW06Y-x6soQItu8byN4Brm6VZK6YawUikZqsNehRxHmd_Q52rd4WJ5cTnLUHSlHNoKzEVOobfvOStE2bkoEceBuwgnjEIgqvFsdEX26lvi7ytkkUad9_Mm4LIs_-MxAnsoop3K0IFMzgq-IQ"
    headers.update({'Authorization': f"Bearer {fireant_bearer_token}"})
    
    FIREANT_API = f"https://restv2.fireant.vn/symbols/{symbol}/full-financial-reports?"
    
    field_to_get = [
        { 'tongcongtaisan': 'totalAssets'},
        { 'taisancodinhhuuhinh': 'tangibleAssets'},
        { 'taisancodinhvohinh': 'intangibleAssets'},
        { 'doanhthuthuan': 'netRevenue'},
        { 'loinhuantruocthue': 'profitBeforeTaxes'},
        { 'loinhuansauthuecuacodongcuacongtyme': 'profitAfterTaxes'},
        { 'tonghangtonkho': 'inventory'},
        { 'nophaitra': 'liabilities'},
        { 'tienvatuongduongtiencuoiky': 'cashAndCashEquivalents'},
        { 'vonchusohuu': 'equity'},
        { 'nonganhan': 'shorttermLiabilities'},
        { 'nodaihan': 'longtermLiabilities'},
        { 'giavonhangban': 'costPrice'},
        { 'khauhaotscd': 'fixedAssetsDepreciation'},
        { 'trongdochiphilaivay': 'lendingCost'},
        { 'vayvanothuetaichinhnganhan': 'shorttermBorrowingsFinancialLeases'},
        { 'vayvanothuetaichinhdaihan': 'longtermBorrowingsFinancialLeases'},
    ]
    
    for year in range(fromYear, toYear+1):
        for quarter in range(1, 4+1):
            if datetime.datetime(year, quarter*3, 1) > datetime.datetime.now():
                continue
            
            print(f"Getting data for {symbol} - {year} - {quarter}")
                  
            quarterBalanceSheet = {}
                                    
            for field in field_to_get:
                quarterBalanceSheet.update({list(field.values())[0] : 0})
             
            # 1  : "candoiketoan"
            # 2  : "ketquakinhdoanh"
            # 3  : "luuchuyentientett"
            # 4  : "luuchuyentientegt"
            for statementType in range(1, 5):                    
                params = {
                    "type": statementType, 
                    "year": year,
                    "quarter": quarter,
                    "limit": 1,
                }
                
                fireant_response = requests.get(FIREANT_API, headers=headers, params=params).json()
                                
                try:
                    if (fireant_response != None):
                        for field_respone in fireant_response:
                            if ('name' not in field_respone): continue
                            
                            field_name =  removeSpace(cleanText(removeVietNameAccent(cleanBullet(cleanRoman(field_respone['name']))))).lower()
                            field_value = field_respone['values'][0]['value'] or 0
                            
                            for field in field_to_get:
                                if field_name in field: quarterBalanceSheet.update({field[field_name]: str(field_value)})
                    
                    quarterBalanceSheet.update({f"year": year})
                    quarterBalanceSheet.update({f"quarter": quarter})
                    
                except:
                    print(f"{symbol} - {year} - {quarter} - {statementType} - {fireant_response}")
                    raise
                
            companyBalanceSheet.append(quarterBalanceSheet)
    return companyBalanceSheet

Lấy chỉ số báo cáo tài chính của công ty

In [ ]:
print(get_balance_sheet())

In [ ]:
mask_active_stock_comapny = (stock_data['status'] == 'listed') & (stock_data['type'] == 'STOCK')
active_company = stock_data[mask_active_stock_comapny]
active_company

In [ ]:
# get balance sheet for all active company and save to csv file
'''
for index, row in active_company.iterrows():
    symbol = row['code']
    company_balance_sheet = get_balance_sheet(symbol)
    
    pd.DataFrame(company_balance_sheet).to_csv(f"./dataset/{symbol}.csv", index=False)
'''

In [ ]:
stock_data.info()

In [ ]:
stock_data['listedDate'] = pd.to_datetime(stock_data['listedDate'], format='%Y-%m-%d')

In [ ]:
active_company_from_2015 = stock_data[(stock_data['listedDate'].dt.year < 2016) & (stock_data['status'] == 'listed') & (stock_data['type'] == 'STOCK')]
active_company_from_2015

In [ ]:
active_company_from_2015 = active_company_from_2015.sample(100)

In [ ]:
"""
for index, row in active_company_from_2015.iterrows():
    symbol = row['code']
    company_balance_sheet = get_balance_sheet(symbol,2017,2022)
    
    pd.DataFrame(company_balance_sheet).to_csv(f"./dataset_from2015/{symbol}.csv", index=False)
"""

### 2. Khám phá dữ liệu 

#### Đọc dữ liệu từ file csv vào data frame

In [ ]:
import glob
import os
from pathlib import Path
import pandas as pd
import matplotlib as plt

In [ ]:
# Add your folder path
folder_path = './dataset'

file_type = 'csv'
seperator =','

files = Path(folder_path).glob('*.csv')

dfs = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

In [ ]:
df

#### Dữ liệu gồm có bao nhiêu dòng và cột?

In [ ]:
num_rows = len(df.axes[0])
num_cols = len(df.axes[1])
print('Row: ',num_rows)
print("Cols: ",num_cols)

#### Mỗi dòng có ý nghĩa gì?

 - Một dòng cho biết báo cáo tài chính của 1 công ty trong 1 quý

#### Dữ liệu có các dòng bị lặp không?

In [ ]:
have_duplicated_rows = False
for i in df.duplicated().items():
    if(i[1] == True):
        have_duplicated_rows = True

In [ ]:
assert have_duplicated_rows == False

- Như vậy không có dòng nào bị lặp

#### Mỗi cột có ý nghĩa gì?

- totalAssets: tổng giá trị tài sản 
- tangibleAssets: tài sản cố định hữu hình
- intangibleAsets: tài sản cố định vô hình
- netRevenue: doanh thu thuần
- profitBeforeTaxes: lợi nhuận trước thuế
- profitAfterTaxes: lợi nhuận sau thuế của cổ đông công ty mẹ
- inventory: tổng hàng tồn kho
- liabilities: nợ phải trả
- cashAndCashEquivalents: tiền mặt và các khoảng tương đương tiền
- equity: vốn chủ sở hữu
- shorttermLiabilities: nợ ngắn hạn
- longtermLiabilities: nợ dài hạn
- costPrice: giá vốn hàng bán
- fixedAssetsDepreciation: khấu hao tài sản cố định
- lendingCost: chi phí lãi vay
- shorttermBorrowingsFinancialLeases: vay nợ tài chính ngắn hạn
- longtermBorrowingsFinancialLeases: vay nợ tài chính dài hạn
- year: năm tài chính
- quarter: quý
- file: mã code của công ty

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [ ]:
col_dtypes = df.dtypes
col_dtypes

### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

In [ ]:
nume_col_df = df.drop(columns='file')
missing_ratio = (nume_col_df.isnull().sum() * 100 / len(nume_col_df))
min = nume_col_df.min()
max = nume_col_df.max()
row_name = ['missing_ratio','min','max']
nume_col_profiles_df = pd.DataFrame([missing_ratio,min,max],index=row_name )
nume_col_profiles_df

### Cột có kiểu dữ liệu dạng không phải numeric, các giá trị được phân bố như thế nào?

In [ ]:
object_col_df = df[["file"]] # tên file là tên của doanh nghiệp
missing_ratio = object_col_df.isnull().sum() * 100 / len(object_col_df)
num_diff_vals = object_col_df.nunique()
diff_vals = [object_col_df[col_name].dropna().unique() for col_name in object_col_df.columns]
r_name = ["missing_ratio", "num_diff_vals", "diff_vals"]

object_col_profiles_df = pd.DataFrame([missing_ratio,num_diff_vals,pd.Series(diff_vals,index=["file"])],index=r_name)
object_col_profiles_df

## 3. Trả lời câu hỏi

#### Câu 1: Công ty nào có tỷ lệ tăng trưởng tốt nhất trong năm 2022?

In [ ]:
df_netRenevue = df[['netRevenue','quarter','year','file']]
df_netRenevue

Chỉ giữ lại các quý cuối của năm

In [ ]:
i = df_netRenevue[(df_netRenevue['quarter'] != 4)].index
df_netRenevue = df_netRenevue.drop(i)

Tính tỷ lệ tăng trưởng của năm 2022 so với năm 2021

In [ ]:
df_netRenevue['Growth Rate']=df_netRenevue.groupby('file')['netRevenue'].pct_change()
df_netRenevue

Xoá các dòng có tỷ lệ tăng trưởng là NaN

In [ ]:
df_netRenevue_gr = df_netRenevue.dropna(subset='Growth Rate')
df_netRenevue_gr

Lấy ra công ty có tỷ lệ tăng trưởng cao nhất

In [ ]:
biggest_growth_rate = df_netRenevue_gr.loc[df_netRenevue_gr['Growth Rate'].idxmax()] 
biggest_growth_rate

In [ ]:
stock_data.loc[stock_data['code'] == biggest_growth_rate['file']]

- Vậy công ty có tỷ lệ tăng trưởng tốt nhất trong năm 2022 là Công ty cổ phần khoáng sản Miền Đông AHP

#

#### 2. Trong 2 năm thu thập, các công ty nào đã bị deslist khỏi sàn chứng khoán, và xếp hạng tăng trưởng doanh thu của họ so với các công ty như thế nào?

Danh sách các công ty đã bị delist

In [ ]:
mask_inactive_stock_comapny = (stock_data['status'] == 'delisted') & (stock_data['type'] == 'STOCK') 
inactive_company = stock_data[mask_inactive_stock_comapny]
inactive_company

In [ ]:
inactive_company['delistedDate'] = pd.to_datetime(inactive_company['delistedDate'], format='%Y-%m-%d')

Danh sách các công ty bị delist trong 2 năm 2021,2022

In [ ]:
inactive_company_21_22 = inactive_company[(inactive_company['delistedDate'].dt.year == 2022) | (inactive_company['delistedDate'].dt.year == 2021)]
inactive_company_21_22

Lấy báo cáo tài chính của các công ty này

In [ ]:
'''
for index, row in inactive_company_21_22.iterrows():
    symbol = row['code']
    company_balance_sheet = get_balance_sheet(symbol)
    
    pd.DataFrame(company_balance_sheet).to_csv(f"./delisted_dataset/{symbol}.csv", index=False)
'''

In [ ]:
folder_path = './delisted_dataset'

file_type = 'csv'
seperator =','

files = Path(folder_path).glob('*.csv')

dfs = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df_2 = pd.concat(dfs, ignore_index=True)

In [ ]:
df_2

Đối với các công ty đã bị delist khỏi sàn thì báo cáo tài chính cũng không có thông tin gì. Ta có thể nhận thấy các công ty này hoạt động không minh bạch nên mới bị delist.

#### 3. Top 10 công ty giàu nhất (có tổng giá trị tài sản lớn nhất và nợ phải trả nhỏ nhất)

In [ ]:
df_sorted = df.sort_values(by=['totalAssets', 'liabilities'], ascending=[False,True])
df_sorted

In [ ]:
i = df_sorted[(df_sorted['year'] == 2021)].index
df_sorted = df_sorted.drop(i)
i = df_sorted[(df_sorted['quarter'] != 4)].index
df_sorted = df_sorted.drop(i)
df_sorted

In [ ]:
top10_df = df_sorted.head(10)
top10_df

In [ ]:
top10_df = top10_df.set_index('file')
top10_df

In [ ]:
top10_df['totalAssets'].plot.bar(xlabel='Tên các công ty', ylabel='Tổng tài sản')

- Dễ dàng nhận thấy các công ty trên đều là Ngân hàng 

#

#### 4. Độ tăng trưởng qua các mốc thời gian của top 3 công ty lên sàn sớm nhất 
#### (Độ tăng trưởng qua thời gian được tính bằng doanh thu thuần của các quý trong 2 năm 2021, 2022)

In [ ]:
active_company['listedDate'] = pd.to_datetime(active_company['listedDate'], format='%Y-%m-%d')
active_company

In [ ]:
active_company = active_company.sort_values(by='listedDate')
active_company

In [ ]:
top3_df = active_company.head(3)
top3_df

In [ ]:
top3_detail_df = df[df['file'].isin(top3_df['code'])]
top3_detail_df

In [ ]:
new = top3_detail_df[['netRevenue','quarter','year','file']].pivot(index=['year','quarter'], columns='file', values='netRevenue')
new

In [ ]:
new.plot(xlabel='Thời gian',ylabel='Doanh thu')

- Tuy là những doanh nghiệp lên sàn sớm nhất nhưng tính hình doanh thu vẫn rất biến động 

#### 5: Xu hướng nợ ngắn hạn của các công ty trong năm 2021, 2022

Ý nghĩa: Thể hiện định hướng kinh doanh của doanh nghiệp và chỉ ra rằng các doanh nghiệp hiện tại có đang đầu tư các mục tiêu ngắn hạn hay không, từ đó phản ánh nền kinh tế nước nhà.

Đầu tiên, xử lý dữ liệu bằng cách chia theo từng năm rồi cộng các quý lại, nghĩa là mình chỉ quan tăm đến nợ ngắn hạn trong quý mà thôi.

In [ ]:
import seaborn as sns

In [ ]:
col_name = ['file', 'shorttermLiabilities', 'profitAfterTaxes', 'year']
info_df = df[col_name]
info_df = info_df.groupby(['file', 'year']).sum().reset_index()
info_df_2021 = info_df[info_df.year==2021]
info_df_2022 = info_df[info_df.year==2022]

Tiếp theo, vẽ biểu đồ cho thấy tương quan giữa nợ ngắn hạn và lợi nhuận sau thuế

In [ ]:
g=sns.FacetGrid(data=info_df, col='year')
g.map(sns.scatterplot, 'profitAfterTaxes', 'shorttermLiabilities', 'file')
g.set_xlabels('Lợi nhuận sau thuế')
g.set_ylabels('Nợ ngắn hạn')
g.fig.subplots_adjust(top=0.7)
g.fig.suptitle('Tương quan giữa lợi nhuận sau thuế và nợ ngắn hạn')

Biểu đồ cho thấy rằng các công ty không có nợ ngắn hạn thường có lợi nhuận ở mức không cao, thể hiện sự an toàn trong đầu tư, ngược lại, các công ty có nợ ngắn hạn sẽ có lợi nhuận đột phá hơn trong năm 2021, nhưng qua năm 2022, có vẻ lợi nhuận của đa số công ty có nợ ngắn hạn chênh lệch không lớn.  
Nhưng, hầu hết các công ty đều có lợi nhuận quanh quẩn mức 0, có nghĩa là trong 2 năm nay, việc tạo ra lợi nhuận bằng các mục tiêu ngắn hạn của các công ty có vẻ không khả quan.
Để kiểm chứng điều đó thì ta cùng thực hiện một số thống kê như sau:

In [ ]:
# Tỷ lệ công ty không có nợ ngắn hạn
count_no_short_Liabilities_2021 = len(info_df_2021[info_df_2021.shorttermLiabilities == 0])/len(info_df_2021)
count_no_short_Liabilities_2022 = len(info_df_2022[info_df_2022.shorttermLiabilities == 0])/len(info_df_2022)
print(f'Ratio of no short liabilities in 2021 is {count_no_short_Liabilities_2021}')
print(f'Ratio of no short liabilities in 2022 is {count_no_short_Liabilities_2022}')

Tỉ lệ này cho ra cỡ 40% các doanh nghiệp không có nợ ngắn hạn, nghĩa là đa số các công ty sẽ chọn không nợ hoặc đầu tư dài hạn hơn.  
Tiếp theo, ta sẽ tìm hiểu về phân bố của nợ ngắn hạn, các công ty có nợ ngắn hạn trong 2 năm này sẽ thường nợ bao nhiêu?

In [ ]:
short_Liabilities = info_df[info_df.shorttermLiabilities > 0]
g=sns.FacetGrid(data=short_Liabilities, col='year')
g.map(sns.histplot, 'shorttermLiabilities', log_scale=True)
g.set_xlabels('Nợ ngắn hạn')
g.fig.subplots_adjust(top=0.7)
g.fig.suptitle('Phân bố nợ ngắn hạn của từng năm')

Mức nợ phân bố chuẩn trong ở lân cận mức 1 tỷ trong năm 2021 và có xu hướng co cụm về mức 1 tỷ trong năm 2022. Việc đầu tư ngắn hạn ở các doanh nghiệp trong 2 năm gần đây đã có sự khởi sắc hơn, khi số lượng tiền vay ở mức 1 tỷ cao hơn trước, thể hiện các doanh nghiệp đã sẵn sàng để quay lại với các mục tiêu trong quý hoặc trong năm.

## Phần 2: Mô Hình hoá Dữ liệu

### Làm việc với Data từ 2015

In [ ]:
folder_path = './dataset_from2015'

file_type = 'csv'
seperator =','

files = Path(folder_path).glob('*.csv')

dfs = list()
for f in files:
    data = pd.read_csv(f)
    # .stem is method for pathlib objects to get the filename w/o the extension
    data['file'] = f.stem
    dfs.append(data)

df_3 = pd.concat(dfs, ignore_index=True)

In [ ]:
df_3

### A. Mô hình hóa dữ liệu và đánh giá mô hình


In [ ]:
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
import plotly.express as px

#### 1. Mô hình dự đoán doanh thu thông qua giá vốn bán hàng. Sử dụng mô hình LinearRegression

Lấy ra các cột 'netRevenue' , 'costPrice' , 'year' , 'quarter' , 'file' 

In [ ]:
df_test = df_3[['netRevenue','costPrice','year','quarter','file']]

Ở đây ta sẽ lấy ra các dòng thuộc quý 4 năm 2022 để làm 1 dataframe dùng để test xem đường hồi quy có fit không

In [ ]:
mask = (df_test['year'] == 2022)
df_test_2 = df_test[mask]
df_test_2

Còn đây là dataframe dùng để trainning

In [ ]:
mask = (df_test['year'] != 2022)
df_test_1 = df_test[mask]
df_test_1

Tính hệ số tương quan

In [ ]:
df_test_1[['netRevenue','costPrice']].corr()

Lưu hai cột netRevenue và costPrice vào 2 biến độc lập

In [ ]:
# Train set
netRevenue = pd.DataFrame(df_test_1['netRevenue'])
costPrice = pd.DataFrame(df_test_1['costPrice'])

Tạo model

In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(costPrice.values ,netRevenue.values)

Hệ số dự đoán của model

In [ ]:
model.coef_

Hệ số chặn y

In [ ]:
model.intercept_

Hệ số xác định của dự đoán.

In [ ]:
model.score(costPrice.values,netRevenue.values)

Lưu hai cột netRevenue và costPrice của df_test_2 vào 2 biến độc lập

In [ ]:
netRevenue_test = pd.DataFrame(df_test_2['netRevenue'])
costPrice_test = pd.DataFrame(df_test_2['costPrice'])

#### Vẽ đồ thị tương quan giữa 2 biến netRevenue và costPrice:
- Các chấm màu xanh thể hiện sự tương quan giữa costPrice và netRevenue trong các quý 1,2,3,4 của năm 2021 và 1,2,3 của năm 2022
- Đường màu đỏ là đường hồi quy tuyến tính dựa trên tập dữ liệu df_test_1
- Các chấm màu đen thể hiện sự tương quan giữa costPrice và netRevenue trong quý 4 năm 2022

In [ ]:
plt.scatter(costPrice,netRevenue)
plt.plot(costPrice , model.predict(costPrice.values), color='red')
plt.scatter(costPrice_test,netRevenue_test, color='black')

**Nhận xét:**
- Đường hồi quy là dự đoán doanh thu nếu tăng giá vốn hàng bán lên.
- Ta có thể thấy đường hồi quy khá fit với các điểm trên đồ thị. Bằng chứng là các điểm hầu như nằm rất gần hoặc năm trên đường hồi quy.
- Có thể thấy đường hồi quy cho ra một kết quả dự đoán khá chính xác, các chấm đen cũng năm rất gần đường hồi quy (sai lệch rất thấp).
- Tuy nhiên, doanh thu thật sự vẫn có trend tăng, nhưng thực tế là khi tăng giá vốn hàng bán, người dùng sẽ khó tiếp cận sản phẩm hơn, nên do đó doanh thu bán hàng của những mặt hàng bán tầm giá vốn đấy doanh thu ít hơn hẳn.

So sánh với LogisticRegression, KNeighborsRegression, DecisionTreeRegression, MLPRegression, BaggingRegression, SVR

In [ ]:
kneighbor = KNeighborsRegressor().fit(costPrice.values, netRevenue.values)
decision = DecisionTreeRegressor().fit(costPrice.values, netRevenue.values)
neural = MLPRegressor().fit(costPrice.values, netRevenue.values)
bagging = BaggingRegressor().fit(costPrice.values, netRevenue.values)
svr = SVR().fit(costPrice.values, netRevenue.values)

In [ ]:
linear_acc = model.score(costPrice.values, netRevenue.values)
kneighbor_acc = kneighbor.score(costPrice.values, netRevenue.values)
decision_acc = decision.score(costPrice.values, netRevenue.values)
neural_acc = neural.score(costPrice.values, netRevenue.values)
bagging_acc = bagging.score(costPrice.values, netRevenue.values)
svr_acc = svr.score(costPrice.values, netRevenue.values)

In [ ]:
fig = px.bar(
    x=['Linear','KNeighbor', 'Decision Tree', 'Neural Network', 'Bagging', 'SVR'],
    y=[linear_acc, kneighbor_acc, decision_acc, neural_acc, bagging_acc, svr_acc],
    color=['Linear','KNeighbor', 'Decision Tree', 'Neural Network', 'Bagging', 'SVR'],
    labels={'x': 'Model', 'y': 'R2_score'},
    title='Model R2 Comparison'
)

In [ ]:
fig.show()

Đánh giá bằng Cross Validation


In [ ]:
from sklearn.model_selection import cross_val_score
cv_costPrice = pd.DataFrame(df_test['costPrice'])
cv_netRevenue = pd.DataFrame(df_test['netRevenue'])
cv_results = cross_val_score(lm, cv_costPrice.values, cv_netRevenue.values, cv=10, error_score='raise')
cv_results.mean()

Do mô hình hồi quy tuyến tính này không có siêu tham số cho nên không thể hiệu chỉnh cho mô hình tốt hơn được

In [ ]:
# Test set
true_netRevenue = pd.DataFrame(df_test_2['netRevenue'])
test_costPrice = pd.DataFrame(df_test_2['costPrice'])

In [ ]:
from sklearn.metrics import mean_squared_error
# Get prediction
predict_netRevenue = lm.predict(test_costPrice.values)
mean_squared_error(predict_netRevenue, true_netRevenue, squared = False)

In [ ]:
plt.scatter(test_costPrice, true_netRevenue)
plt.plot(test_costPrice, lm.predict(test_costPrice.values), color='red')

Mô hình hồi quy tuyến tính cho điểm số cỡ $0.44*10^{12}$ trong độ đo RMSE 

### Mô hình với prophet

In [ ]:
from prophet import Prophet
from prophet.plot import plot_plotly,plot_components_plotly

In [ ]:
df_test_1

In [ ]:
def calc_date_time(row):
    month = row['quarter'] * 3
    year = row['year']
    return f"30/{month}/{year}"

In [ ]:
df_new = df_test_1.copy()
df_new['date'] = df_new.apply(calc_date_time,axis=1)

df_new_2 = df_test_2.copy()
df_new_2['date'] = df_new_2.apply(calc_date_time,axis=1)

In [ ]:
df_train = df_new[['netRevenue','date']]
df_train

In [ ]:
df_train = df_train.rename(columns={'netRevenue':'y','date':'ds'})
df_train.ds = pd.to_datetime(df_train.ds)
df_train

In [ ]:
m = Prophet()
m.fit(df_train)

In [ ]:
future = m.make_future_dataframe(periods=90)
forecast = m.predict(future)
future

In [ ]:
plot_plotly(m,forecast)

In [ ]:
# Make cv set
from prophet.diagnostics import performance_metrics, cross_validation
df_cv = cross_validation(m, horizon = '365 days')
df_cv

In [ ]:
rmse = performance_metrics(df_cv)['rmse']
rmse

In [ ]:
import itertools
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.2, 0.4, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 2.0, 3.0, 4.0, 5.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(df_train)  # Fit model with given params
    df_cv = cross_validation(m, horizon='365 days')
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses

In [ ]:
tuning_results[tuning_results.rmse == tuning_results.rmse.min()]

In [ ]:
new_prophet = Prophet(changepoint_prior_scale=0.01, seasonality_prior_scale=5)

In [ ]:
# Test set
df_test = df_new_2[['netRevenue','date']].rename(columns={'netRevenue':'y','date':'ds'})
df_test.ds = pd.to_datetime(df_test.ds)
df_test

In [ ]:
# Pre-train
new_prophet.fit(df_train) 

In [ ]:
yhat = m.predict(df_test[['ds']]).yhat
yhat
y = df_test.y
y
#RMSE
mean_squared_error(yhat, y, squared = False)

Điểm RMSE của Prophet cho ra cỡ $0.18 * 10^{13}$, tệ hơn điểm của Hồi quy tuyến tính, cho thấy rằng thị trường tài chính trong năm 2022 khá khác so với các năm trước (do dịch covid chẳng hạn), dẫn đến việc sử dụng mô hình này trong tình huống này kém hiệu quả hơn